 Import bibliotecas
 

In [106]:
!pip install influxdb

In [107]:
import csv 
from influxdb import InfluxDBClient
from datetime import datetime
import pandas as pd
#from pandas.io.parsers.readers import read_csv
from pandas import read_csv
import random
import time
import numpy as np 
import random
import math

Conexão com o banco

In [160]:
import json
client = InfluxDBClient(host='bitsaas-dns-vm-iot.eastus2.cloudapp.azure.com', port=8086, ssl=True, verify_ssl=True)
client.switch_database('db_dados_sensores')
results = client.query('SELECT * FROM "bit_leaf_teste"')
lista = []
total = 0 #tempo total monitorado, em ms
ligado = 0 #tempo total ligado, em ms
inicio = 0 #instante de início, em ms
termino = 0 #instante de término, em ms
consumo = 0 #spent
for i in results:
    for result in i:
          if result['time']:
            total += (result['updtimestart'] - result['updtimeend']) #soma o tempo total monitorado
            t = datetime.fromtimestamp(result['updtimestart']/1000) #transforma o updtimestart de ms para s, divide por mil e transforma em timestamp
            result['on-off']= not result['on-off']
            if result['on-off']: #verificar se a bomba está ligada
              ligado+= 1000 #add 1000 ms ao tempo da bomba ligada
            if inicio ==0: #verifica se não tem registro do início de monitoramento
              inicio = result['updtimestart'] #registra o início do monitoramento
            if termino < result['updtimestart']: #verifica se o término atual é menor que o registro atual
              termino = result['updtimestart'] #registra a última verificação
            result['spent'] = random.random()
            consumo += result['spent'] #registra o consumo 
            blueprint = {
                #"device": result['device'],
                "on-off": result['on-off'],
                "sensor": result['sensor'],
                "updtimeend" : result['updtimeend'],
                "updtimestart" : result['updtimestart'],
                "spent" : result ['spent'],
                "subtotal": (result['updtimestart'] - result['updtimeend']),
                #Total": total,
                "Tempo ligado (ms)": ligado,
                "Data": datetime.strftime(t, "%d/%m/%Y"),
                #"Data": datetime.strftime(t, "%d/%m/%Y %H:%M:%S"),#exibe o timestamp em formato datetime (data completa legível)
                "Dia": datetime.strftime(t,"%a"),
                "Hora": datetime.strftime(t, "%H:%M:%S")
                }
            lista.append(blueprint)        
#print(blueprint)
print(termino)
print(inicio)
print(ligado)
print(consumo)

1644424740000
1644418800000
1010000
490.91008644870203


####          Transforma dados extraídos em um DataFrame. 

In [161]:
df =pd.DataFrame(lista)
df.head(100) 

,on-off,sensor,updtimeend,updtimestart,spent,subtotal,Tempo ligado (ms),Data,Dia,Hora
0,True,water bomb,1644418740000,1644418800000,0.121787,60000,1000,09/02/2022,Wed,15:00:00
1,True,water bomb,1644418800000,1644418860000,0.780326,60000,2000,09/02/2022,Wed,15:01:00
2,True,water bomb,1644418860000,1644418920000,0.807622,60000,3000,09/02/2022,Wed,15:02:00
3,True,water bomb,1644418920000,1644418980000,0.556255,60000,4000,09/02/2022,Wed,15:03:00
4,True,water bomb,1644418980000,1644419040000,0.628688,60000,5000,09/02/2022,Wed,15:04:00
...,...,...,...,...,...,...,...,...,...,...
95,True,water bomb,1644423840000,1644423900000,0.810852,60000,96000,09/02/2022,Wed,16:25:00
96,True,water bomb,1644423900000,1644423960000,0.732163,60000,97000,09/02/2022,Wed,16:26:00
97,True,water bomb,1644423960000,1644424020000,0.505471,60000,98000,09/02/2022,Wed,16:27:00
98,True,water bomb,1644424020000,1644424080000,0.134256,60000,99000,09/02/2022,Wed,16:28:00


In [148]:
df.groupby('spent') # agrupa os campos selecionados nos df. 

In [147]:
df.groupby('spent').count() # total em que o device ficou ligado consumindo energia. 

,on-off,sensor,updtimeend,updtimestart,subtotal,On,Data
spent,,,,,,,
0.001559,1,1,1,1,1,1,1
0.001828,1,1,1,1,1,1,1
0.003418,1,1,1,1,1,1,1
0.004225,1,1,1,1,1,1,1
0.009226,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
0.995804,1,1,1,1,1,1,1
0.995855,1,1,1,1,1,1,1
0.996497,1,1,1,1,1,1,1


In [162]:
print(df)

      on-off      sensor     updtimeend   updtimestart     spent  subtotal  \
0       True  water bomb  1644418740000  1644418800000  0.121787     60000   
1       True  water bomb  1644418800000  1644418860000  0.780326     60000   
2       True  water bomb  1644418860000  1644418920000  0.807622     60000   
3       True  water bomb  1644418920000  1644418980000  0.556255     60000   
4       True  water bomb  1644418980000  1644419040000  0.628688     60000   
...      ...         ...            ...            ...       ...       ...   
1005    True  water bomb  1644424440000  1644424500000  0.051267     60000   
1006    True  water bomb  1644424500000  1644424560000  0.806763     60000   
1007    True  water bomb  1644424560000  1644424620000  0.210115     60000   
1008    True  water bomb  1644424620000  1644424680000  0.424701     60000   
1009    True  water bomb  1644424680000  1644424740000  0.082417     60000   

      Tempo ligado (ms)        Data  Dia      Hora  
0         

In [150]:
print(lista)

[{'on-off': True, 'sensor': 'water bomb', 'updtimeend': 1644418740000, 'updtimestart': 1644418800000, 'spent': 0.6952438243442689, 'subtotal': 60000, 'On': 1000, 'Data': 'Wed 09/02/2022 15:00:00'}, {'on-off': True, 'sensor': 'water bomb', 'updtimeend': 1644418800000, 'updtimestart': 1644418860000, 'spent': 0.05868045792050092, 'subtotal': 60000, 'On': 2000, 'Data': 'Wed 09/02/2022 15:01:00'}, {'on-off': True, 'sensor': 'water bomb', 'updtimeend': 1644418860000, 'updtimestart': 1644418920000, 'spent': 0.581745359200905, 'subtotal': 60000, 'On': 3000, 'Data': 'Wed 09/02/2022 15:02:00'}, {'on-off': True, 'sensor': 'water bomb', 'updtimeend': 1644418920000, 'updtimestart': 1644418980000, 'spent': 0.8679736726541096, 'subtotal': 60000, 'On': 4000, 'Data': 'Wed 09/02/2022 15:03:00'}, {'on-off': True, 'sensor': 'water bomb', 'updtimeend': 1644418980000, 'updtimestart': 1644419040000, 'spent': 0.6157379954464962, 'subtotal': 60000, 'On': 5000, 'Data': 'Wed 09/02/2022 15:04:00'}, {'on-off': Tru

In [32]:
strdate= 1646470800

#df = pd.DataFrame(strdate)
#df['updtimeend'] = pd.to_datetime(df['updtimeend'], format='%Y%m%d%H%M%S')

#datetimeobj=datetime.strptime(strdate,"%Y-%m-%d %H:%M:%S")

tempo = datetime.fromtimestamp(strdate)
print(tempo)

#adc coluna nova

2022-03-05 09:00:00


#### Formula que calcula os dados para previsão de consumo: 

In [ ]:
'''consumo = 9.43 * (Kw_boia+Kw_bb2) # consumo da semana convertido em R$ tarifa Enell 1.05
print('kw',consumo)
custo = consumo *( Kw_boia + Kw_bb2)*tarifa
print (f'R$:',custo) # consumo médio diário. Quando o device esp32 e acionado sua boia para controle de nível de água do reservatório também e acionada.  